In [19]:
import dill
import pandas as pd
import altair as alt
import numpy as np

In [3]:
d_all_EOD = dill.load(open('data/d_all_EOD.pkd','rb'))
for ticker in d_all_EOD.keys():
    d_all_EOD[ticker] = d_all_EOD[ticker].reset_index(drop=True)

In [31]:
def find_low(ticker, date):
    df_truth = d_all_EOD[ticker]
    df1 = df_truth[df_truth.date==date]
    index = df1.index[0]
    return df_truth.iloc[index-1]['low']

def find_high(ticker,date):
    df_truth = d_all_EOD[ticker]
    df1 = df_truth[df_truth.date==date]
    index = df1.index[0]
    return df_truth.iloc[index+1]['high']

def calc_pe(prediction,true_value):
    return 100*(prediction - true_value)/true_value

def calc_ape(prediction,true_value):
    return 100*np.abs(prediction - true_value)/true_value
    
def calc_mpe(d):
    l_temp =[]
    for ticker, dates in d.items():
        for date, df in dates.items():
            true_low = find_low(ticker,date)
            true_high = find_high(ticker,date)
            l_temp.append({'date': date, 'ticker': ticker,
                        'pe_low': calc_pe(d[ticker][date]['low'], true_low), 
                        'pe_high': calc_pe(d[ticker][date]['high'], true_high),
                        'ape_low': calc_ape(d[ticker][date]['low'], true_low),
                        'ape_high': calc_ape(d[ticker][date]['high'], true_high)})
    df_results = pd.DataFrame(l_temp)
    
    print('MPE Low:',df_results.pe_low.mean())
    print('MPE High:',df_results.pe_high.mean())
    print('MPE Overall:',df_results[['pe_low','pe_high']].mean()[0])
    print('MAPE Low:',df_results.ape_low.mean())
    print('MAPE High:',df_results.ape_high.mean())
    print('MAPE Overall:',df_results[['ape_low','ape_high']].mean()[0])
    
    return df_results.sort_values('date').reset_index(drop=True)

def show_results(predictions_path):
    df = calc_mpe(dill.load(open(predictions_path,'rb')))
    return alt.Chart(df_pe,title='PE for Low and High').mark_line().encode(x='date',y='pe_low')&\
alt.Chart(df_pe).mark_line().encode(x='date',y='pe_high')

### CVAE predictions - baseline

In [32]:
show_results('data/predictions_cvae83.pkd')

MPE Low: 1.640174301338967
MPE High: 2.4722017017146234
MPE Overall: 1.640174301338967
MAPE Low: 11.94763298855777
MAPE High: 11.546397276446879
MAPE Overall: 11.94763298855777


alt.VConcatChart(...)

### CVAE Predictions - With Sentiment

In [33]:
show_results('data/predictions_cvae83_sent.pkd')

MPE Low: 0.43418693994337965
MPE High: 1.4903526315177495
MPE Overall: 0.43418693994337965
MAPE Low: 11.109080723798684
MAPE High: 10.924856218318737
MAPE Overall: 11.109080723798684


alt.VConcatChart(...)

### NN Predictions with Sentiment

In [34]:
show_results('data/predictions_baseline83_sent.pkd')

MPE Low: -0.03842297908281038
MPE High: 0.20082292371585528
MPE Overall: -0.03842297908281038
MAPE Low: 2.2057702933255943
MAPE High: 1.5927511357294062
MAPE Overall: 2.2057702933255943


alt.VConcatChart(...)